<a href="https://colab.research.google.com/github/borundev/DNN_Lectures/blob/master/Bayesian_Pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Bayesian Pool Game

Alice and Bob play a game where Carol comes in randomly pics a location on a pool table but does not tell Alice and Bob about it. Then Alice and Bob take turns shooting the ball and if it lands on the left of Carol's mark Alice scores a point and if it ends on the right Bob scores a point. As a game its bit contrived but it is useful to demonstrate the effectiveness of Bayesian analysis.

If Carol's initial location is given by $p \in [0,1]$ then on each shot Alice wins with a probability $p$ and Bob win's with a probability $(1-p)$. The first one to score 6 points wins the game.

So the question is if after $K$ games Alice has won $n_A$ and Bob has won $n_B = K-n_A$ games, what are the odds of Bob winning the tournament? 

A Frequentist approach would look like this. The point estimate of $p$ is 

$$
\hat p = \frac{n_A}{K}
$$

The maximum number of games that are required to be played to get a decision is 11. Now Bob can win by scoring at least $6-n_B$ in the remaining $11-K$ games so

$$
P(B|D)_F = \sum_{\tilde n_B=6-n_B}^{11-K} \phantom{a}^{11-K} C_{\tilde n_B} (\frac{n_B}{K})^{\tilde n_B} (\frac{K-n_B}{K})^{11-K-\tilde n_B}
$$

The Bayesian approach is different, and as we will confirm, correct. What we want is the probability of Bob's win given the data.

$$
P(B|D) = \int P(B|p,D) P(p|D) dp \\
= \int P(B|p,D) \frac{P(D|p) P(p)}{P(D)} dp \\
= \frac{ \int P(B|p,D) P(D|p) P(p) dp}{\int P(D|p) P(p) dp}
$$

Let's work out the denominator first

$$
P(p) = 1 
$$
so we do not need to worry about it. We have

$$
P(D|p) = \phantom{a}^K C_{n_B} (1-p)^{n_B} p^{K-n_B}
$$
giving us
$$
P(D) = \phantom{a}^K C_{n_B} \int_0^1 p^{K-n_B} (1-p)^{n_B} dp  \\
= \phantom{a}^K C_{n_B} \frac{ n_B! (K-n_B)!}{K+1!} \\
=\frac{1}{K+1}
$$
where we have used the defintion of Beta functions and their relations with Gamma functions.

What a cute little result. Think about why this is the case.

Now let's work on the neumerator. We have
$$
P(B|p,D)P(D|p) = \sum_{\tilde n_B=6-n_B}^{11-K} \phantom{a}^{11-K} C_{\tilde n_B} \phantom{a}^{K} C_{n_B} (1-p)^{\tilde n_B+n_B} p^{11-\tilde n_B -n_B}
$$

giving us

$$
P(B|D) = \frac{(K+1) \phantom{a}^{K} C_{n_B}}{12!} \sum_{\tilde n_B=6-n_B}^{11-K} \phantom{a}^{11-K} C_{\tilde n_B}  (n_B+ \tilde n_B)! (11-n_B- \tilde n_B)! 
$$

In [5]:
import numpy as np
from scipy.special import gamma

In [6]:
def fac(a):
    return gamma(a+1)

def comb(a,b):
    return fac(a)/fac(b)/fac(a-b)


def pB(K,nb):
    if nb>K:
        return 0
    z=(K+1)*comb(K,nb)/fac(12)
    return z*np.sum([comb(11-K,nbt)*fac(nb+nbt)*fac(11-nb-nbt) for nbt in range(6-nb,12-K)])

def pF(K,nb):
    if nb>K:
        return 0
    p=1-nb/K
    return np.sum([comb(11-K,nbt) * (1-p)**nbt * (p)**(11-K-nbt) for nbt in range(6-nb,12-K)])

In [7]:
K=8
nb=3
pB(K,nb),pF(K,nb)

(0.0909090909090909, 0.052734375)

In [9]:
import numpy as np
np.random.seed(0)

# play 100000 games with randomly-drawn p, between 0 and 1
p = np.random.random(10000000)

# each game needs at most 11 rolls for one player to reach 6 wins
rolls = np.random.random((11, len(p)))

# count the cumulative wins for Alice and Bob at each roll
Alice_count = np.cumsum(rolls < p, 0)
Bob_count = np.cumsum(rolls >= p, 0)

# sanity check: total number of wins should equal number of rolls
total_wins = Alice_count + Bob_count
assert np.all(total_wins.T == np.arange(1, 12))
print("(Sanity check passed)")

(Sanity check passed)


In [10]:
results=[]
for K in range(1,12):
    for nb in range(K+1):
        good_games=Bob_count[K-1]==nb
        bob_won=np.sum(Bob_count[:,good_games][10] >= 6)
        mc_prob = bob_won * 1. / good_games.sum()

        # compute the probability
        results.append([K,nb,pB(K,nb),pF(K,nb),mc_prob])

In [11]:
import pandas as pd

In [12]:
df=pd.DataFrame(results)

In [13]:
df.columns=['K','nb','Bayesian','Frequentist','MC']

In [14]:
df=df.set_index(['K','nb'])

We look at only rows where all values are not 0 or 1

In [17]:
df2=df[~np.stack([np.isclose(df,0).all(1),np.isclose(df,1).all(1)],1).any(1)]

In [18]:
df2

Bayesian  Frequentist        MC
K  nb                                 
1  0   0.227273     0.000000  0.227231
   1   0.772727     1.000000  0.772754
2  0   0.090909     0.000000  0.090904
   1   0.500000     0.500000  0.500017
   2   0.909091     1.000000  0.909032
3  0   0.030303     0.000000  0.030300
   1   0.272727     0.087944  0.273068
   2   0.727273     0.912056  0.726722
   3   0.969697     1.000000  0.969813
4  0   0.007576     0.000000  0.007599
   1   0.121212     0.012878  0.121435
   2   0.500000     0.500000  0.500000
   3   0.878788     0.987122  0.878504
   4   0.992424     1.000000  0.992425
5  0   0.001082     0.000000  0.001056
   1   0.040043     0.001600  0.040166
   2   0.283550     0.179200  0.284041
   3   0.716450     0.820800  0.716016
   4   0.959957     0.998400  0.960024
   5   0.998918     1.000000  0.998931
6  1   0.007576     0.000129  0.007586
   2   0.121212     0.045267  0.121537
   3   0.500000     0.500000  0.500472
   4   0.878788     0.954733  0.878431
   5   0.992424     0.999871  0.992458
7  2   0.030303     0.006664  0.030335
   3   0.272727     0.213661  0.273299
   4   0.727273     0.786339  0.727164
   5   0.969697     0.993336  0.969692
8  3   0.090909     0.052734  0.091266
   4   0.500000     0.500000  0.500448
   5   0.909091     0.947266  0.908779
9  4   0.227273     0.197531  0.227897
   5   0.772727     0.802469  0.772519
10 5   0.500000     0.500000  0.500202